In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

Training the CNN

In [2]:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.9
log_interval = 10

#Normalizing the data given the global mean (0.1307) and standard deviation (0.3081)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

In [3]:
trainset = datasets.MNIST("/files/", train=True, download=True, transform=transform)

train_loader = DataLoader(trainset, batch_size=batch_size_train, shuffle=True)

testset = datasets.MNIST("/files/", train=False, download=True, transform=transform)

test_loader = DataLoader(testset, batch_size=batch_size_test, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_numpy.cpp:174.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, 5)
    self.conv2 = nn.Conv2d(10, 20, 5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x)

mnist_cnn = Net()

optimizer = optim.SGD(mnist_cnn.parameters(), lr=learning_rate, momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i * len(train_loader.dataset) for i in range (n_epochs + 1)]

In [5]:
def train(epoch): 
  mnist_cnn.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = mnist_cnn(data)

    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(output, target)
    loss.backward()
    optimizer.step()

    #displaying the results
    if batch_idx % log_interval == 2:
      print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset)))

In [7]:
def test():
  mnist_cnn.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      output = mnist_cnn(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()

  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)

  print("\nTest set: Avg. loss: {:.4f}, Accuracy : {}/{} ({:.0f}%)\n".format(test_loss, correct, len(test_loader.dataset), 100.*correct/len(test_loader.dataset)))

In [8]:
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [128/60000 (0%)]	Loss: 2.315636
Train Epoch: 1 [768/60000 (1%)]	Loss: 2.278350
Train Epoch: 1 [1408/60000 (2%)]	Loss: 2.282228
Train Epoch: 1 [2048/60000 (3%)]	Loss: 2.197255
Train Epoch: 1 [2688/60000 (4%)]	Loss: 1.901575
Train Epoch: 1 [3328/60000 (6%)]	Loss: 1.570577
Train Epoch: 1 [3968/60000 (7%)]	Loss: 1.176932
Train Epoch: 1 [4608/60000 (8%)]	Loss: 1.034968
Train Epoch: 1 [5248/60000 (9%)]	Loss: 1.098745
Train Epoch: 1 [5888/60000 (10%)]	Loss: 0.880842
Train Epoch: 1 [6528/60000 (11%)]	Loss: 0.700734
Train Epoch: 1 [7168/60000 (12%)]	Loss: 0.724953
Train Epoch: 1 [7808/60000 (13%)]	Loss: 0.664565
Train Epoch: 1 [8448/60000 (14%)]	Loss: 0.564879
Train Epoch: 1 [9088/60000 (15%)]	Loss: 0.646881
Train Epoch: 1 [9728/60000 (16%)]	Loss: 0.500868
Train Epoch: 1 [10368/60000 (17%)]	Loss: 0.618689
Train Epoch: 1 [11008/60000 (18%)]	Loss: 0.797737
Train Epoch: 1 [11648/60000 (19%)]	Loss: 0.633440
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.573847
Train Epoch: 1 [12928/60000

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.1061, Accuracy : 9656/10000 (97%)

Train Epoch: 2 [128/60000 (0%)]	Loss: 0.359546
Train Epoch: 2 [768/60000 (1%)]	Loss: 0.259199
Train Epoch: 2 [1408/60000 (2%)]	Loss: 0.167773
Train Epoch: 2 [2048/60000 (3%)]	Loss: 0.292370
Train Epoch: 2 [2688/60000 (4%)]	Loss: 0.245583
Train Epoch: 2 [3328/60000 (6%)]	Loss: 0.391367
Train Epoch: 2 [3968/60000 (7%)]	Loss: 0.199534
Train Epoch: 2 [4608/60000 (8%)]	Loss: 0.356784
Train Epoch: 2 [5248/60000 (9%)]	Loss: 0.192347
Train Epoch: 2 [5888/60000 (10%)]	Loss: 0.137778
Train Epoch: 2 [6528/60000 (11%)]	Loss: 0.526871
Train Epoch: 2 [7168/60000 (12%)]	Loss: 0.255832
Train Epoch: 2 [7808/60000 (13%)]	Loss: 0.203329
Train Epoch: 2 [8448/60000 (14%)]	Loss: 0.325793
Train Epoch: 2 [9088/60000 (15%)]	Loss: 0.252838
Train Epoch: 2 [9728/60000 (16%)]	Loss: 0.334179
Train Epoch: 2 [10368/60000 (17%)]	Loss: 0.325661
Train Epoch: 2 [11008/60000 (18%)]	Loss: 0.552098
Train Epoch: 2 [11648/60000 (19%)]	Loss: 0.210430
Train Epoch: 2 [12

#### Generating Image Labels

In [9]:
import shutil
import os

for img in os.listdir("../input/wgan-generated-mnist/Generated_MNIST"):
    image = Image.open(f"../input/wgan-generated-mnist/Generated_MNIST/{img}").convert("L")
    
    #Normalizing the images
    transform = transforms.ToTensor()
    tensor = transform(image)
    
    normalize = transforms.Normalize((0.1307,), (0.3081,))
    img_tensor = normalize(tensor)
    
    resize =  transforms.Resize(28)
    image_tensor = resize(img_tensor)
    
    label = mnist_cnn(image_tensor[None,...])
    label = torch.argmax(label)
    
    if(os.path.isdir(f"./{label}") == False):
        os.mkdir(f"./{label}")
    
    shutil.copy(f"../input/wgan-generated-mnist/Generated_MNIST/{img}", f"./{label}/{img}")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
